## 生成对应Json表情包文件

### Artalk

In [ ]:
import os
import json

def generate_artalk_emojis(base_url, emoji_folders, output_path='.json/artalk-emoji.json', origin=None):
    """
    生成 Artalk 表情包 JSON 文件
    
    参数:
        base_url: 基础 URL，例如 'https://fastly.jsdelivr.net/gh/willow-god/owo'
        emoji_folders: 包含表情包的文件夹列表
        output_path: 输出 JSON 文件的路径
        origin: 原始表情包数据，包含网址到名称的映射关系
    """
    result = []
    
    # 创建网址到名称的映射字典
    url_to_key_map = {}
    if origin:
        for group in origin:
            for item in group.get("items", []):
                if "val" in item and "key" in item:
                    # 去除可能存在的反引号
                    val = item["val"].replace("`", "").strip()
                    url_to_key_map[val] = item["key"]
    
    for folder in emoji_folders:
        if not os.path.isdir(folder):
            print(f"警告: {folder} 不是一个文件夹，已跳过")
            continue
        
        emoji_group = {
            "name": folder,
            "type": "image",
            "items": []
        }
        
        # 获取文件夹中的所有文件
        files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
        
        for file in files:
            # 获取文件名（不带扩展名）
            filename_without_ext = os.path.splitext(file)[0]
            
            # 构建完整URL
            full_url = f"{base_url}/{folder}/{file}"
            
            # 检查是否在原始数据中存在该URL，如果存在则使用原始key
            key = url_to_key_map.get(full_url, filename_without_ext)
            
            # 创建表情项
            emoji_item = {
                "key": key,
                "val": full_url
            }
            
            emoji_group["items"].append(emoji_item)
        
        if emoji_group["items"]:  # 只添加非空的表情组
            result.append(emoji_group)
    
    # 确保输出目录存在
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 写入 JSON 文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"表情包 JSON 文件已生成: {output_path}")

if __name__ == "__main__":
    base_url = "https://fastly.jsdelivr.net/gh/willow-god/owo"
    folders = ["liushen", "blobcat", "linedog", "bilibili"]
    origin_json = ".json/artalk-emoji.json"
    output = ".json/artalk-emoji.json"
    
    # 加载原始表情包数据（如果存在）
    origin_data = None
    try:
        with open(origin_json, 'r', encoding='utf-8') as f:
            origin_data = json.load(f)
        print(f"已加载原始表情包数据: {origin_json}")
    except FileNotFoundError:
        print(f"未找到原始表情包数据文件: {origin_json}")
    except json.JSONDecodeError:
        print(f"原始表情包数据文件格式错误: {origin_json}")
    
    generate_artalk_emojis(base_url, folders, output, origin_data)

已加载原始表情包数据: .json/artalk-emoji.json
表情包 JSON 文件已生成: .json/artalk-emoji.json


### Twikoo

In [5]:
import os
import json

def generate_twikoo_emojis(base_url, emoji_folders, output_path='.json/twikoo-emojis.json', origin=None):
    """
    生成 Twikoo 表情包 JSON 文件
    
    参数:
        base_url: 基础 URL，例如 'https://fastly.jsdelivr.net/gh/willow-god/owo'
        emoji_folders: 包含表情包的文件夹列表
        output_path: 输出 JSON 文件的路径
        origin: 原始表情包数据，包含网址到名称的映射关系
    """
    result = {}
    
    # 创建网址到名称的映射字典
    url_to_text_map = {}
    if origin:
        for category, data in origin.items():
            if "container" in data:
                for item in data["container"]:
                    if "icon" in item and "text" in item:
                        # 从 icon 中提取 URL
                        import re
                        url_match = re.search(r"src=['\"]([^'\"]+)['\"]", item["icon"])
                        if url_match:
                            url = url_match.group(1).replace("`", "").strip()
                            url_to_text_map[url] = item["text"]
    
    for folder in emoji_folders:
        if not os.path.isdir(folder):
            print(f"警告: {folder} 不是一个文件夹，已跳过")
            continue
        
        # 为每个文件夹创建一个类别
        category_name = folder
        # 可以根据文件夹名称自定义类别显示名称
        display_names = {
            "blobcat": "可爱猫",
            "linedog": "线条狗",
            "bilibili": "小电视",
            "liushen": "流沙",
            # 可以添加更多映射
        }
        
        category_display_name = display_names.get(folder, folder)
        
        emoji_category = {
            "type": "image",
            "container": []
        }
        
        # 获取文件夹中的所有文件
        files = [f for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]
        
        for file in files:
            # 获取文件名（不带扩展名）
            filename_without_ext = os.path.splitext(file)[0]
            
            # 构建完整URL
            full_url = f"{base_url}/{folder}/{file}"
            
            # 检查是否在原始数据中存在该URL，如果存在则使用原始text
            text = url_to_text_map.get(full_url, f"{filename_without_ext}")
            
            # 创建表情项
            emoji_item = {
                "text": text,
                "icon": f"<img src='{full_url}'>"
            }
            
            emoji_category["container"].append(emoji_item)
        
        if emoji_category["container"]:  # 只添加非空的表情组
            result[category_display_name] = emoji_category
    
    # 确保输出目录存在
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    # 写入 JSON 文件
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(result, f, ensure_ascii=False, indent=2)
    
    print(f"Twikoo 表情包 JSON 文件已生成: {output_path}")

if __name__ == "__main__":
    base_url = "https://fastly.jsdelivr.net/gh/willow-god/owo"
    folders = ["liushen", "blobcat", "linedog", "bilibili"]
    origin_json = ".json/twikoo.json"
    output = ".json/twikoo-emoji.json"
    
    # 加载原始表情包数据（如果存在）
    origin_data = None
    try:
        with open(origin_json, 'r', encoding='utf-8') as f:
            origin_data = json.load(f)
        print(f"已加载原始表情包数据: {origin_json}")
    except FileNotFoundError:
        print(f"未找到原始表情包数据文件: {origin_json}")
    except json.JSONDecodeError:
        print(f"原始表情包数据文件格式错误: {origin_json}")
    
    generate_twikoo_emojis(base_url, folders, output, origin_data)

未找到原始表情包数据文件: .json/twikoo.json
Twikoo 表情包 JSON 文件已生成: .json/twikoo-emoji.json


## 图片处理部分

### 压缩指定文件夹列表中的所有图片到指定尺寸

In [4]:
import os
from PIL import Image
import glob

def resize_images(folders, max_size=200, output_suffix="_resized"):
    """
    将指定文件夹中的所有图片进行等比缩小，保持最大边等于指定尺寸，并保留透明区域
    
    参数:
        folders: 包含图片的文件夹列表
        max_size: 缩放后的最大边长（像素）
        output_suffix: 输出文件的后缀，如果为None则覆盖原文件
    
    返回:
        处理成功的图片数量
    """
    supported_formats = ['.png', '.jpg', '.jpeg', '.gif', '.webp']
    processed_count = 0
    
    for folder in folders:
        if not os.path.isdir(folder):
            print(f"警告: {folder} 不是一个有效的文件夹，已跳过")
            continue
            
        print(f"处理文件夹: {folder}")
        
        # 获取文件夹中所有支持格式的图片
        image_files = []
        for ext in supported_formats:
            image_files.extend(glob.glob(os.path.join(folder, f"*{ext}")))
            image_files.extend(glob.glob(os.path.join(folder, f"*{ext.upper()}")))
        
        for img_path in image_files:
            try:
                # 打开图片
                img = Image.open(img_path)
                
                # 获取原始尺寸
                width, height = img.size
                
                # 如果图片已经小于或等于目标尺寸，则跳过
                if width <= max_size and height <= max_size:
                    print(f"跳过 {img_path} (已经小于或等于目标尺寸)")
                    # continue
                
                # 计算缩放比例
                ratio = min(max_size / width, max_size / height)
                new_width = int(width * ratio)
                new_height = int(height * ratio)
                
                # 等比缩放图片
                resized_img = img.resize((max_size, max_size), Image.LANCZOS)
                
                # 确定输出路径
                if output_suffix:
                    filename, ext = os.path.splitext(img_path)
                    output_path = f"{filename}{output_suffix}{ext}"
                else:
                    output_path = img_path
                
                # 保存图片，保留原始格式和透明度
                if img.mode == 'RGBA':
                    # 保留透明通道
                    resized_img.save(output_path, format=img.format, quality=95)
                else:
                    # 普通图片
                    resized_img.save(output_path, format=img.format, quality=95)
                
                processed_count += 1
                print(f"已处理: {img_path} -> {output_path} ({width}x{height} -> {new_width}x{new_height})")
                
            except Exception as e:
                print(f"处理图片失败 {img_path}: {e}")
    
    print(f"处理完成! 共处理 {processed_count} 张图片")
    return processed_count

# 示例用法
if __name__ == "__main__":
    # 要处理的文件夹列表
    folders_to_process = ["liushen", "bilibili"] # 其他均为动态图像，不太好处理
    
    # 设置最大尺寸为150像素
    max_size = 150
    
    # 处理图片，生成带有_resized后缀的新文件
    resize_images(folders_to_process, max_size, None)
    
    # 如果想要覆盖原文件，可以将output_suffix设为None
    # resize_images(folders_to_process, max_size, None)

处理文件夹: liushen
跳过 liushen\liushen-angry.png (已经小于或等于目标尺寸)
已处理: liushen\liushen-angry.png -> liushen\liushen-angry.png (146x150 -> 146x150)
跳过 liushen\liushen-chowdown.png (已经小于或等于目标尺寸)
已处理: liushen\liushen-chowdown.png -> liushen\liushen-chowdown.png (141x150 -> 141x150)
跳过 liushen\liushen-cold.png (已经小于或等于目标尺寸)
已处理: liushen\liushen-cold.png -> liushen\liushen-cold.png (149x150 -> 149x150)
跳过 liushen\liushen-confused.png (已经小于或等于目标尺寸)
已处理: liushen\liushen-confused.png -> liushen\liushen-confused.png (150x146 -> 150x146)
跳过 liushen\liushen-congratulation.png (已经小于或等于目标尺寸)
已处理: liushen\liushen-congratulation.png -> liushen\liushen-congratulation.png (150x146 -> 150x146)
跳过 liushen\liushen-cry.png (已经小于或等于目标尺寸)
已处理: liushen\liushen-cry.png -> liushen\liushen-cry.png (150x148 -> 150x148)
跳过 liushen\liushen-cutehold.png (已经小于或等于目标尺寸)
已处理: liushen\liushen-cutehold.png -> liushen\liushen-cutehold.png (150x148 -> 150x148)
跳过 liushen\liushen-dislike.png (已经小于或等于目标尺寸)
已处理: liushen\liushen-dislik